## How2link 数据集抽取

In [42]:
from tqdm import tqdm
import json
import os 
json_path = "/home/kanghenglin/streamit/Howto-Interlink7M_subset_w_all_clips_val.json"
datas = json.load(open(json_path,'r'))
video_caption = {}

for idx, data in enumerate(tqdm(datas[:1000],total=1000)):     
    try:
        sample = data['clips'][1]
    except:
        sample = data['clips'][0]
        
    s3_path = sample['clip_path']
    tos_path = s3_path.replace("s3://kanelin/interlink7m/samples/data/","tos:kanelin/interlink7m/") + ".mp4"
    local_dir = "/data/streamlit_source/keyframe_sapmle/how2link"
    local_path = os.path.join(local_dir,f"{idx+1}.mp4")
    caption = sample['caption']

    video_caption[idx+1] = caption
    
    !rclone copy "$tos_path" "$local_dir"
    os.rename(os.path.join(local_dir,s3_path.split("/")[-1]+".mp4"),local_path)

# 保存video_caption字典
json.dump(video_caption,open("/data/streamlit_source/keyframe_sapmle/how2link/caption.json",'w'))


100%|██████████| 1000/1000 [00:00<00:00, 312541.28it/s]


## Webvid 数据集抽取

In [43]:
from tqdm import tqdm
import json
import os
json_path = "/data/streamlit_source/rmwm_webvid_QA_train_clean_train.json"
datas = json.load(open(json_path,'r'))
video_caption = {}

for idx in tqdm(range(1000)):
    
    prefix = "oss:vision-language-data/video-data/webvid10m/process_videos"
    tos_path = os.path.join(prefix, datas['image'][str(idx)])
    
    local_dir = "/data/streamlit_source/keyframe_sapmle/webvid"
    local_path = os.path.join(local_dir,f"{idx+1}.mp4")
    caption = datas['value'][str(idx)]
    
    video_caption[idx+1] = caption
    
    !rclone copy "$tos_path" "$local_dir"
    os.rename(os.path.join(local_dir,tos_path.split("/")[-1]),local_path)
    

# 保存video_caption字典
json.dump(video_caption,open("/data/streamlit_source/keyframe_sapmle/webvid/caption.json",'w'))

100%|██████████| 1000/1000 [00:00<00:00, 249705.54it/s]


## HDVila 数据集抽取

In [ ]:
from tqdm import tqdm
import json
import os 
json_path = "/data/streamlit_source/path_to_output_hd-3m3.json"
datas = json.load(open(json_path,'r'))
video_caption = {}

for idx, data in enumerate(tqdm(datas[:1000],total=1000)):
    
    prefix = "oss:vision-language-data/video-data/hd130m/process_videos/"    
    tos_path = os.path.join(prefix, data['video'])
    
    local_dir = "/data/streamlit_source/keyframe_sapmle/hdvila"
    local_path = os.path.join(local_dir,f"{idx+1}.mp4")
    caption = data['caption']
    
    video_caption[idx+1] = caption
    
    !rclone copy "$tos_path" "$local_dir"
    os.rename(os.path.join(local_dir,tos_path.split("/")[-1]),local_path)
    
# 保存video_caption字典
json.dump(video_caption,open("/data/streamlit_source/keyframe_sapmle/hdvila/caption.json",'w'))

## InternVid 数据集抽取

In [ ]:
import pandas as pd
import os
from tqdm import tqdm
import shutil
import json

video_caption = {}
df = pd.read_json('/data/streamlit_source/raw_json/InternVid-10M-FLT-INFO.jsonl', lines=True)
video_dir = "/mnt/shared-storage/tenant/hypertext/kanelin/data/internvid/InternVId-FLT/"
meta_data = df[:1000]
meta_data = meta_data.to_dict()
for idx in tqdm(range(1000)):
    file_name = os.path.join(video_dir,"_".join([meta_data['YoutubeID'][idx],meta_data['Start_timestamp'][idx],meta_data['End_timestamp'][idx]]) + ".mp4")
    caption = meta_data['Caption'][idx]
    shutil.copy(file_name, "/data/streamlit_source/keyframe_sapmle/internvid")
    os.rename(f"/data/streamlit_source/keyframe_sapmle/internvid/{file_name.split('/')[-1]}", f"/data/streamlit_source/keyframe_sapmle/internvid/{idx+1}.mp4")
    video_caption[idx+1] = caption

json.dump(video_caption,open("/data/streamlit_source/keyframe_sapmle/internvid/caption.json",'w'))


## Momentor 数据集抽取

In [ ]:
from tqdm import tqdm
import json
import os 
import shutil

print('Loading Json ...')
json_path = "/mnt/shared-storage/tenant/hypertext/kanelin/data/Moment-10M/json/Moment-10M_0.json"
datas = json.load(open(json_path,'r'))
print('Start Processing ...')
video_ids = list(datas.keys())
video_caption = {}

for idx, video_id in enumerate(tqdm(video_ids[:1000],total=1000)):
    
    prefix = "/mnt/shared-storage/tenant/hypertext/kanelin/data/Moment-10M/video"
    video_path = os.path.join(prefix, video_id + ".mp4")
    
    if not os.path.exists(video_path):
        continue
    
    local_dir = "/data/streamlit_source/keyframe_sapmle/momentor"
    local_path = os.path.join(local_dir,f"{idx+1}.mp4")
    
    caption = datas[video_id]
    
    video_caption[idx+1] = caption
    
    # !cp "$video_path" "$local_dir"
    shutil.copy(video_path, local_dir)
    
    os.rename(os.path.join(local_dir,video_path.split("/")[-1]),local_path)
    
# 保存video_caption字典
json.dump(video_caption,open("/data/streamlit_source/keyframe_sapmle/momentor/caption.json",'w'))